In [6]:
from youtube_transcript_api import YouTubeTranscriptApi
import os
from openai import OpenAI
import requests
from dotenv import load_dotenv
import streamlit as st
import subprocess
import torch

In [101]:
load_dotenv()

True

In [3]:
def check_gpu():
    try:
        import torch
        if torch.cuda.is_available():
            return {
                "Device": torch.cuda.get_device_name(0),
                "CUDA Version": torch.version.cuda,
                "Memory Total (GB)": round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 2),
                "Memory Available (GB)": round(torch.cuda.mem_get_info()[0] / (1024**3), 2)
            }
    except:
        pass
    return None

In [9]:
print(torch.cuda.is_available())

False


In [8]:
print(torch.backends.cudnn.enabled)

True


In [4]:
check_gpu

<function __main__.check_gpu()>

In [2]:
try:
    subprocess.check_output('nvidia-smi')
    print('Nvidia GPU detected!')
except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('No Nvidia GPU in system!')

Nvidia GPU detected!


In [90]:
def get_youtube_transcript(video_url: str):
    # extract video ID from URL
    video_id = video_url.split("v=")[1]

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # combine all text entries into a single string
        full_transcript = " ".join([entry['text'] for entry in transcript])

        return full_transcript
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

In [105]:
def summarize_transcript():
    st.title("YouTube Transcript Summarizer")

    video_url = st.text_input("Input YouTube URL")

    # Add sliders for paramaters
    col1, col2 = st.columns(2)
    with col1:
        temperature = st.slider("Temperature (Randomness)", 0.0, 1.0, 0.5, 0.01)
    with col2:
        presence_penalty = st.slider("Presence Penalty (Likelihood of discussing new topics)", 0.0, 1.0, 0.05, 0.01)

    # Create a button to trigger the start of the summarization
    if st.button("Summarize"):
        if not video_url:
            st.warning("Please enter a YouTube URL")
            return
        
        transcript = get_youtube_transcript(video_url)

        if transcript:
            st.info("Generating summary...")

            url = "https://api.perplexity.ai/chat/completions"

            payload = {
                "model": "sonar-pro",
                "messages": [
                    {"role": "user", "content": f"Summarize this transcript into high quality and organized notes:{transcript}"}
                ],
                "max_tokens": 1000,
                "temperature": temperature,
                "presence_penalty": presence_penalty

            }

            headers = {
                "Authorization": f"Bearer {os.getenv("PERPLEXITY_API_KEY")}",
                "Content-Type": "application/json"
            }

            try:
                response = requests.post(url, json=payload, headers=headers)
                response_json = response.json()

                if "choices" in response_json and len(response_json['choices'] > 0):
                    
                    summary = response.json().get("choices")[0].get("message").get("content")
                    st.markdown(summary)
                else:
                    st.error(f"Error in API response: {response_json}")
            except Exception as e:
                st.error(f"Error calling Perplexity API: {e}")

In [106]:
summarize_transcript()

2025-03-31 11:40:43.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 11:40:43.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar